In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Buscar los comentarios
comment_elements = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
comments = [comment.get_text(strip=True) for comment in comment_elements]

# Imprimir comentarios
for idx, comment in enumerate(comments, 1):
    print(f"Comentario {idx}: {comment}")


Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.


In [2]:
with open('pagina_html.txt', 'w', encoding='utf-8') as file:
    file.write(soup.prettify())

In [9]:
# Ejemplo de uso de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Ruta de tu ChromeDriver
driver_path = "D:\\Proyecto Web Scrapping\\chromedriver.exe"  # Nota: usa doble backslash "\\" en Windows
service = Service(driver_path)

# Configurar ChromeDriver con el servicio
driver = webdriver.Chrome(service=service)
driver.get("https://www.google.com")
print("Título de la página:", driver.title)
driver.quit()

Título de la página: Google


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import time

# Configura el controlador de Chrome con Service
driver_path = 'D:\\Proyecto Web Scrapping\\chromedriver.exe'  # Ruta completa a tu ChromeDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# URL de la página
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'
driver.get(url)

# Espera inicial para cargar la página
time.sleep(5)

# Paso 1: Haz clic en el botón "Mostrar todas las opiniones" para abrir la ventana emergente
try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.show-more-click[data-testid="see-more"]')
    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)  # Desplaza hasta el botón
    time.sleep(1)
    driver.execute_script("arguments[0].click();", load_more_button)  # Haz clic en el botón
    time.sleep(3)  # Espera para que la ventana emergente de comentarios se abra
except (NoSuchElementException, ElementClickInterceptedException) as e:
    print("No se encontró o no se pudo hacer clic en el botón de 'Mostrar todas las opiniones'.", e)

# Paso 2: Encuentra el contenedor específico de comentarios
try:
    comments_container = driver.find_element(By.CSS_SELECTOR, 'section[data-testid="reviews-desktop"]')
except NoSuchElementException:
    print("No se encontró el contenedor de comentarios.")
    driver.quit()
    exit()

# Paso 3: Desplazamiento progresivo dentro del contenedor de comentarios para cargar todos los bloques de comentarios
scroll_pause_time = 2
last_height = driver.execute_script("return arguments[0].scrollHeight", comments_container)

while True:
    # Desplaza dentro del contenedor de comentarios
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_container)
    time.sleep(scroll_pause_time)

    # Calcula la nueva altura de desplazamiento del contenedor
    new_height = driver.execute_script("return arguments[0].scrollHeight", comments_container)
    if new_height == last_height:
        # Si la altura no cambia, ya no hay más comentarios para cargar
        break
    last_height = new_height

# Paso 4: Extrae los comentarios de la página completamente cargada con BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
comment_elements = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
comments = [comment.get_text(strip=True) for comment in comment_elements]

# Imprime los comentarios extraídos
for idx, comment in enumerate(comments, 1):
    print(f"Comentario {idx}: {comment}")

# Cierra el navegador
driver.quit()


Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.
